IEC ArcFlash calculator How to use:

Run the calculator by pressing shift and enter
Add value to the box and hit enter
Note: To make sure programme is running correctly enter the values in IEEE standard 

In [4]:
import math
import numpy as np

'''
intermediate arcing current coefficients are below
'''
VCB_600 = [-0.04287, 1.035, -0.083, 0, 0,
           -4.783e-09, 1.962e-06, -0.000229, 0.003141, 1.092]

VCBB_600 = [-0.017432, 0.98, -0.05, 0, 0,
            -5.767E-09, 2.524E-06, -0.00034, 0.01187, 1.013]

HCB_600 = [0.054922, 0.988, -0.11, 0, 0,
           -5.382E-09, 2.316E-06, -0.000302, 0.0091, 0.9725]

VOA_600 = [0.043785, 1.04, -0.18, 0, 0,
           -4.783E-09, 1.962E-06, -0.000229, 0.003141, 1.092]

HOA_600 = [0.111147, 1.008, -0.24, 0, 0,
           -3.895E-09, 1.641E-06, -0.000197, 0.002615, 1.1]

Voc = float(input('Enter system voltage in kV: '))  # 0.480
I_bf = float(input('Enter fault current in kA: '))  # 45
G = float(input('Enter gap between electrodes in mm: '))  # 32
D = float(input('Enter working distance in mm: '))  # 609.6
width = float(input('Enter width of container in mm: '))  # 610
height = float(input('Enter height of container in mm:'))  # 610
depth = float(input('Enter depth of container in mm:'))  # 254

print('How are the conductors arranged?')
print('1. vertical conductors/electrodes inside a metal box/enclosure (VCB)')
print('2. vertical conductors/electrodes terminated in an insulating barrier inside a metal box/ enclosure (VCBB)')
print('3. horizontal conductors/electrodes inside a metal box/enclosure (HCB)')
print('4. vertical conductors/electrodes in open air (VOA)')
print('5. horizontal conductors/electrodes in open air (HOA)')
a = float(input('1, 2, 3, 4, 5'))

def Iarc_600(VCB_600, I_bf, G):
    Iarc_600 = 10**(VCB_600[0] +
                    VCB_600[1]*math.log10(I_bf) +
                    VCB_600[2]*math.log10(G)) * (VCB_600[3]*I_bf**6 +
                                                 VCB_600[4]*I_bf**5 +
                                                 VCB_600[5]*I_bf**4 +
                                                 VCB_600[6]*I_bf**3 +
                                                 VCB_600[7]*I_bf**2 +
                                                 VCB_600[8]*I_bf +
                                                 VCB_600[9])
    return Iarc_600

if (a == 1): 
    Iarc_600 = (Iarc_600(VCB_600, I_bf, G))
    A = 4
    B = 20
elif (a == 2):
    Iarc_600 = (Iarc_600(VCBB_600, I_bf, G))
    A = 10
    B = 24
elif (a == 3):
    Iarc_600 = (Iarc_600(HCB_600, I_bf, G))
    A = 10
    B = 22
elif (a == 4):
    Iarc_600 = (Iarc_600(VOA_600, I_bf, G))
elif (a == 5):
    Iarc_600 = (Iarc_600(HOA_600, I_bf, G))
else:
    print('input not correct')


def Iarc(Voc, Iarc_600, I_bf):
    Iarc = 1/(np.sqrt(
           ((0.6/Voc)**2) *
           ((1/Iarc_600**2) -
            ((0.6**2-Voc**2)
            /
            (0.6**2*I_bf**2)))))
    return Iarc

def width1equ(width, A, B):
    width1 = 660.4+(width-660.4)*((Voc+A)/B)*25.4**-1
    return width1

def height1equ(height, A, B):
    height1 = 660.4+(height-660.4)*((Voc+A)/B)*25.4**-1
    return width1

Iarc = Iarc(Voc, Iarc_600, I_bf)
print('Current of arc(kA) is: ')
print(Iarc)
print('Check the protection characteristics to find the disconnection time!')
T = float(input(''))


box_shallow = True

if (Voc < 0.600 or (height < 508 and width < 508) or depth <= 203.2):
    box_shallow = True

else: 
    print('box is typical')
    width1 = 20
    height1 = 20
    box_shallow = False
    


if (box_shallow == True and (height <= 660.4 and width <= 660.4)):
    width1 = 0.03937 * width
    height1 = 0.03937 * height
elif (box_shallow == True and (height > 660.4 and height <= 1244.6 and width >660.4 and width <= 1244.6)):
    if (a == 1):
        height1 = height * 0.03937
        width1 = width1equ(width,A,B)
    else:
        height1 = height1equ(height,A,B)
        width1 = width1equ(width,A,B)
elif (box_shallow == True and (height > 1244.6 and width > 1244.6)):
    if (a == 1):
        height1 = 49
        width1 = width1equ(1244.6,A,B)
    else:
        height1 = height1equ(1244.6,A,B)
        width1 = width1equ(1244.6,A,B)

EES = (height1 + width1)/2  # Equivalent enclosure size 

if (a == 1 and box_shallow == True):
    b1 = -0.000302
    b2 = 0.03441
    b3 = 0.4325
elif (a==2 and box_shallow == True):
    b1 = -0.0002976
    b2 = 0.032
    b3 = 0.479
elif (a==3 and box_shallow == True):
    b1 = -0.0001923
    b2 = 0.01935
    b3 = 0.6899
elif (a==1 and box_shallow == False):
    b1 = 0.002222
    b2 = -0.02556 
    b3 = 0.6222
elif (a==2 and box_shallow == False):
    b1 = -0.002778
    b2 = 0.1194
    b3 = -0.2778
elif (a==3 and box_shallow == False):
    b1 = -0.0005556
    b2 = 0.03722
    b3 = 0.4778

if (box_shallow == True):
    CF = b1*EES**2 +b2*EES+b3
elif(box_shallow == False):
    CF = 1/(b1*EES**2 +b2*EES+b3)






def inter_energy(T, k, G, Iarc_600, I_bf, D, Iarc, CF):
    E_600 = (12.552/50) * T * 10 ** (k[0] + k[1]*math.log10(G) +
                                 ((k[2]*Iarc_600)/(k[3]*I_bf**7 +
                                                   k[4]*I_bf**6 +
                                                   k[5]*I_bf**5 +
                                                   k[6]*I_bf**4 +
                                                   k[7]*I_bf**3 +
                                                   k[8]*I_bf**2 +
                                                   k[9]*I_bf)) +
                                 k[10]*math.log10(I_bf) +
                                 k[11]*math.log10(D) +
                                 k[12]*math.log10(Iarc) +
                                 math.log10(1/CF))
    return E_600

energy_VCB = [0.753364, 0.566, 1.752636, 0, 0, -4.783e-09, 0.000001962, -0.000229,0.003141, 1.092, 0, -1.598, 0.957]
energy_VCBB = [3.068459, 0.26, -0.098107, 0, 0, -5.767E-09, 0.000002524, -0.00034, 0.01187, 1.013, -0.06, -1.809, 1.19]
energy_HCB = [4.073745, 0.344, -0.370259, 0, 0, -5.382E-09, 0.000002316, -0.000302, 0.0091, 0.9725, 0, -2.03, 1.036]
energy_VOA = [0.679294, 0.746, 1.222636, 0, 0, -4.783E-09, 0.000001962, -0.000229, 0.003141, 1.092, 0, -1.598, 0.997]
energy_HOA = [3.470417, 0.465, -0.261863, 0, 0, -3.895E-09, 0.000001641, -0.000197, 0.002615, 1.1, 0, -1.99, 1.04]


if (a==1):
    E600=inter_energy(T, energy_VCB, G, Iarc_600, I_bf, D, Iarc, CF)
elif (a==2):
    E600=inter_energy(T,energy_VCBB,G,Iarc_600,I_bf,D,Iarc,CF)
elif (a==3):
    E600=inter_energy(T,energy_HCB,G,Iarc_600,I_bf,D,Iarc,CF)
elif (a==4):
    E600=inter_energy(T,energy_VOA,G,Iarc_600,I_bf,D,Iarc,CF)
elif (a==5):
    E600=inter_energy(T,energy_HOA,G,Iarc_600,I_bf,D,Iarc,CF)
print(E600)

def AFB(k, G, Iarc_600, I_bf, Iarc, CF, T):
    AFB600 = 10 ** ((k[0] + k[1]*math.log10(G) +
              ((k[2]*Iarc_600)/(k[3]*I_bf**7 +
                                   k[4]*I_bf**6 +
                                   k[5]*I_bf**5 +
                                   k[6]*I_bf**4 +
                                   k[7]*I_bf**3 +
                                   k[8]*I_bf**2 +
                                   k[9]*I_bf)) +
                 k[10]*math.log10(I_bf) +
                 k[12]*math.log10(Iarc) +
                 math.log10(1/CF) -
                 math.log10(20/T))/-k[11])
    return AFB600


if (a==1):
    AFB600 = AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==2):
    AFB600 = AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==3):
    AFB600 = AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==4):
    AFB600 = AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==5):
    AFB600 = AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)


Iarc_var_VCB = [0, -0.0000014269, 0.000083137, -0.0019382, 0.022366, -0.12645, 0.30226]
Iarc_var_VCBB = [1.138e-06, -6.0287e-05, 0.0012758, -0.013778, 0.080217, -0.24066, 0.33524]
Iarc_var_HCB = [0, -3.097e-06, 0.00016405, -0.0033609, 0.033308, -0.16182, 0.34627]
Iarc_var_HOA = [9.5606E-07, -5.1543E-05, 0.0011161, -0.01242, 0.075125, -0.23584, 0.33696]
Iarc_var_VOA = [0, -3.1555e-06, 0.0001682, -0.0034607, 0.034124, -0.1599, 0.34629]

def VarCF(k, Voc):
    VarCf = k[0] * Voc**6 + k[1] * Voc**5 + k[2] * Voc**4 + k[3] * Voc**3 + k[4] * Voc ** 2 + k[5] * Voc + k[6]
    return VarCf

if (a==1):
    varCf = VarCF(Iarc_var_VCB, Voc)
elif (a==2):
    varCf = VarCF(Iarc_var_VCBB, Voc)
elif (a==3):
    varCf = VarCF(Iarc_var_HCB, Voc)
elif (a==4):
    varCf = VarCF(Iarc_var_VOA, Voc)
elif (a==5):
    varCf = VarCF(Iarc_var_HOA, Voc)

Iarc = 1/(np.sqrt(((0.6/Voc)**2)*((1/Iarc_600**2)-(0.6**2-Voc**2)/(0.6**2*I_bf**2))))*(1-0.5*varCf)
print('Current of arc(kA) is: ')
print(Iarc)
T = float(input('Check protection characteristics to check the new disconnection time:'))

if (a==1):
    adjusted_E600=inter_energy(T, energy_VCB, G, Iarc_600, I_bf, D, Iarc, CF)
elif (a==2):
    adjusted_E600=inter_energy(T,energy_VCBB,G,Iarc_600,I_bf,D,Iarc,CF)
elif (a==3):
    adjusted_E600=inter_energy(T,energy_HCB,G,Iarc_600,I_bf,D,Iarc,CF)
elif (a==4):
    adjusted_E600=inter_energy(T,energy_VOA,G,Iarc_600,I_bf,D,Iarc,CF)
elif (a==5):
    adjusted_E600=inter_energy(T,energy_HOA,G,Iarc_600,I_bf,D,Iarc,CF)
print('Energy of arc(J/cm2) is : ')
print(adjusted_E600)

if (a==1):
    adjusted_AFB600=AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==2):
    adjusted_AFB600=AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==3):
    adjusted_AFB600=AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==4):
    adjusted_AFB600=AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
elif (a==5):
    adjusted_AFB600=AFB(energy_VCB, G, Iarc_600, I_bf,Iarc,CF,T)
print('Arc flash boundary(mm) is: ')
print(adjusted_AFB600)

Enter system voltage in kV:  0.48
Enter fault current in kA:  45
Enter gap between electrodes in mm:  32
Enter working distance in mm:  609.6
Enter width of container in mm:  610
Enter height of container in mm: 610
Enter depth of container in mm: 254


How are the conductors arranged?
1. vertical conductors/electrodes inside a metal box/enclosure (VCB)
2. vertical conductors/electrodes terminated in an insulating barrier inside a metal box/ enclosure (VCBB)
3. horizontal conductors/electrodes inside a metal box/enclosure (HCB)
4. vertical conductors/electrodes in open air (VOA)
5. horizontal conductors/electrodes in open air (HOA)


1, 2, 3, 4, 5 1


Current of arc(kA) is: 
28.79327345188356
Check the protection characteristics to find the disconnection time!


 61.3


11.584954873360502
Current of arc(kA) is: 
25.24439950704917


Check protection characteristics to check the new disconnection time: 319


Energy of arc(J/cm2) is : 
53.15632836899013
Arc flash boundary(mm) is: 
2668.918763074401
